##Download the whole ImageNet

In [ ]:
!pip install datasets Pillow
!huggingface-cli login
from datasets import load_dataset
ds = load_dataset("imagenet-1k", split = "validation")

##Download a subset of ImageNet

In [9]:
!pip install kaggle
!mkdir /content/kaggle
!cp kaggle.json /content/kaggle/
!chmod 600 /content/kaggle/kaggle.json
!kaggle datasets download ifigotin/imagenetmini-1000
!unzip /content/imagenetmini-1000.zip

## Import packages

In [ ]:
!pip install pot

In [28]:
import numpy as np
import torch
from torchvision.models import resnet50, efficientnet_b3, EfficientNet_B3_Weights
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

from torchvision.models._api import WeightsEnum
from torch.hub import load_state_dict_from_url

from sklearn.metrics.pairwise import pairwise_distances
import ot

## Use EfficientNet to get prediction distribution

Load images

In [54]:
root_dir = '/content/imagenet-mini'

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

images = ImageFolder('/content/imagenet-mini/train', transform=transform)

# Use DataLoader for easy iteration
batch_size = 1
ds_dataloader = DataLoader(images, batch_size=1, shuffle=False)

Get pretrained EfficientNet_B3

In [ ]:
def get_state_dict(self, *args, **kwargs):
    kwargs.pop("check_hash")
    return load_state_dict_from_url(self.url, *args, **kwargs)
WeightsEnum.get_state_dict = get_state_dict

model = efficientnet_b3(weights=EfficientNet_B3_Weights.IMAGENET1K_V1)
model.eval()

Get the prediction distribution of images

In [75]:
l_pred_dist = []
with torch.no_grad():
    i = 1
    for images, labels in ds_dataloader:
        logits = model(images)
        probabilities = torch.nn.functional.softmax(logits, dim=1)
        l_pred_dist.append(probabilities.numpy().flatten())
        i += 1
        if i == 1000:
          break

## Similarity matrix between images by wasserstein distance

In [77]:
C = np.ones((1000, 1000)) # need to be updated

def wasserstein(u,v):
  ot_emd = ot.emd(u, v, C)
  return np.sum(ot_emd * C)

D = pairwise_distances(l_pred_dist, metric=wasserstein)

## Image Ordering